### Verificación del Entorno MLOps

Este documento te guiará para validar que todos los servicios de tu infraestructura MLOps (Jupyter, Spark, MLflow, Kafka y Airflow) están activos y se comunican correctamente.

#### 1\. Verificar Contenedores Activos

Primero, asegúrate de que todos los contenedores definidos en tu `docker-compose.yml` estén corriendo.

  * **Comando:**

    ```bash
    docker ps
    ```

  * **Resultado Esperado:** Deberías ver una lista con contenedores para los siguientes servicios (los nombres pueden variar ligeramente según tu configuración):

      * `python_ml_stack` (Jupyter/Spark/MLflow)
      * `postgres_db`
      * `kafka`
      * `zookeeper`
      * `airflow_webserver`
      * `airflow_scheduler`

Si algún contenedor no está en la lista o su estado es `Exited`, revisa los logs con `docker logs <nombre_contenedor>`.

#### 2\. Acceder a las Interfaces Web (UIs)

Verifica que puedes acceder a las interfaces gráficas de los servicios desde tu navegador.

  * **JupyterLab:**

      * URL: `http://localhost:8888`
      * Token: El que definiste en tu `Dockerfile` (`hola-mundo` según tu configuración actual) o el que aparece en los logs de inicio.
      * **Prueba:** Intenta crear un nuevo notebook.

  * **Spark Master UI:**

      * URL: `http://localhost:8080`
      * **Prueba:** Verifica que aparezca el Worker conectado y que haya recursos disponibles (Cores/Memory).

  * **MLflow UI:**

      * URL: `http://localhost:5000`
      * **Prueba:** Deberías ver la interfaz de MLflow. Intenta crear un experimento de prueba desde un notebook (ver sección 4).

  * **Airflow Webserver:**

      * URL: `http://localhost:8081`
      * Credenciales: `admin` / `admin` (según las variables de entorno configuradas).
      * **Prueba:** Inicia sesión y verifica que puedes ver la lista de DAGs de ejemplo.

#### 3\. Probar la Conexión con Kafka

Para validar que Kafka y Zookeeper funcionan y son accesibles desde tu entorno de Python (Jupyter), ejecuta el siguiente script en un notebook de Jupyter.

  * **Prueba (Kafka):**

In [4]:
from kafka import KafkaProducer, KafkaConsumer
import json

# Configuración
KAFKA_TOPIC = 'test_topic'
KAFKA_BOOTSTRAP_SERVERS = 'kafka:29092' # Usamos el nombre del servicio definido en docker-compose

# 1. Crear Productor
try:
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    print("Productor Kafka conectado exitosamente.")

    # Enviar mensaje
    message = {'mensaje': 'Hola desde Jupyter!', 'id': 1}
    producer.send(KAFKA_TOPIC, message)
    producer.flush()
    print(f"Mensaje enviado a '{KAFKA_TOPIC}': {message}")

except Exception as e:
    print(f"Error conectando Productor: {e}")

# 2. Crear Consumidor
try:
    consumer = KafkaConsumer(
        KAFKA_TOPIC,
        bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
        auto_offset_reset='earliest', # Leer desde el principio
        enable_auto_commit=True,
        group_id='my-group',
        value_deserializer=lambda x: json.loads(x.decode('utf-8')),
        consumer_timeout_ms=5000 # Esperar 5 segundos y salir si no hay mensajes
    )
    print("Consumidor Kafka conectado exitosamente.")

    # Leer mensaje
    for msg in consumer:
        print(f"Mensaje recibido: {msg.value}")
        break # Solo queremos probar que llega uno

except Exception as e:
    print(f"Error conectando Consumidor: {e}")

Productor Kafka conectado exitosamente.
Mensaje enviado a 'test_topic': {'mensaje': 'Hola desde Jupyter!', 'id': 1}
Consumidor Kafka conectado exitosamente.
Mensaje recibido: {'mensaje': 'Hola desde Jupyter!', 'id': 1}


#### 4\. Probar Spark + MLflow

Verifica que Spark pueda procesar datos y registrar experimentos en MLflow. Puedes reusar o adaptar el código de tu `Cuaderno_1.ipynb`, asegurándote de que la URI de MLflow sea correcta.

  * **Notebook de Prueba (Spark + MLflow):**

In [5]:
from pyspark.sql import SparkSession
import mlflow

# 1. Inicializar Spark en modo LOCAL
# Usamos "local[*]" para usar todos los núcleos del contenedor sin necesitar red
spark = SparkSession.builder \
    .appName("Test_Spark_MLflow") \
    .master("local[*]") \
    .config("spark.driver.host", "127.0.0.1") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")

# 2. Configurar MLflow
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Test_Experiment")

# 3. Registrar un run simple
with mlflow.start_run():
    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 0.85)
    print("Run registrado en MLflow")

Spark Version: 3.5.1
Run registrado en MLflow


#### 5\. Probar Airflow (Opcional)

Para verificar que Airflow está escaneando correctamente tu carpeta `dags`, crea un archivo de prueba simple en tu carpeta local `dags/`.
  * **Archivo `dags/test_dag.py`:**


```python
from airflow import DAG
from airflow.operators.bash import BashOperator
from datetime import datetime

with DAG(
    'test_dag_simple',
    start_date=datetime(2023, 1, 1),
    schedule_interval=None,
    catchup=False
) as dag:

    t1 = BashOperator(
        task_id='print_date',
        bash_command='date',
    )

    t2 = BashOperator(
        task_id='echo_hello',
        bash_command='echo "Hola Airflow!"',
    )

    t1 >> t2
```

In [6]:
from airflow import DAG
from airflow.operators.bash import BashOperator
from datetime import datetime

with DAG(
    'test_dag_simple',
    start_date=datetime(2023, 1, 1),
    schedule_interval=None,
    catchup=False
) as dag:

    t1 = BashOperator(
        task_id='print_date',
        bash_command='date',
    )

    t2 = BashOperator(
        task_id='echo_hello',
        bash_command='echo "Hola Airflow!"',
    )

    t1 >> t2

ModuleNotFoundError: No module named 'airflow'

  * **Validación:**

    1.  Guarda el archivo.
    2.  Espera unos segundos y refresca la UI de Airflow (`http://localhost:8081`).
    3.  Deberías ver `test_dag_simple` en la lista.
    4.  Actívalo (toggle "On") y ejecútalo manualmente (botón "Play").
    5.  Verifica que las tareas se pongan en verde (Success).
